<a href="https://colab.research.google.com/github/yueguo1997/SNA_recommender_system/blob/main/GNN_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>